In [2]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder
from cosapp.drivers import NonLinearSolver, RunOnce, MonteCarlo, RunSingleCase
from cosapp.utils.distributions import Normal
import sys
sys.path.append('./model')
from Earth import Earth
import json
import numpy as np

earth = Earth("earth")

l = 2 #Rocket's length on the plot
dt = 0.05 #Time-step


from cosapp.utils.distributions import Normal
from cosapp.drivers import MonteCarlo
from cosapp.recorders import DataFrameRecorder

In [3]:

LOAD = True

if LOAD:
    with open("./include/init_rocket/rocket_dict.json", "r") as f:
        rocket_dict = json.load(f)
    l = rocket_dict['tube_length'] + rocket_dict['nose_length'] #Rocket's length on the plot
    angz = - rocket_dict['rocket_launch_angle']
    # Load the thrust.txt
    thrust = rocket_dict['motor']['samples']
    with open("model/Utility/thrust.txt", "w") as f:
        for i in range(len(thrust[0])):
            f.write(", ".join([str(point[i]) for point in thrust]))
            if i < len(thrust[0]) - 1:
                f.write("\n")

init = {
    'Traj.r' : np.array([-(l/2)*np.sin(angz), 0., (l/2)*np.cos(angz)]),
    'Rocket.Kin.v' : np.array([0,0,0]),
    'Rocket.Kin.ar' : 'initrot',
    'Rocket.Kin.av' : np.zeros(3),
    'Para.DynPar.r1' : np.array([0., 0., l/2]),
    'Para.DynPar.r2' : np.array([0., 0., l/2]),
    'Para.DynPar.v1' : np.array([0,0,0]),
    'Para.DynPar.v2' : np.array([0,0,0]),
    'Traj.ParaDepStatus': False
}
# rocket_dict parameters
if LOAD:
    init = {**init, 
            'Traj.r' : np.array([0,0, rocket_dict['rocket_cog'][0]]),
            'Para.DynPar.r1' : np.array([0., 0., rocket_dict['rocket_cog'][0]]),
            'Para.DynPar.r2' : np.array([0., 0., rocket_dict['rocket_cog'][0]]),
            'Rocket.CG': rocket_dict['rocket_cog'][0],
            'Rocket.l' : l,
            'Rocket.Mass.m' : rocket_dict['rocket_mass'],
            'Rocket.Mass.m0' : rocket_dict['rocket_mass'],
            'Rocket.Mass.Dm' : rocket_dict['rocket_prop_weight']/thrust[-1][0],
            'Rocket.Mass.lastEngineTime' : thrust[-1][0],
            'Rocket.Mass.I0_geom' : [rocket_dict['rocket_inertia'][i][i] for i in range(3)],

            'Rocket.Aero.Coefs.ln' : rocket_dict['nose_length'],
            'Rocket.Aero.Coefs.d' : 2*rocket_dict['tube_radius'],
            'Rocket.Aero.Coefs.NFins' : rocket_dict['fins_number'],
            'Rocket.Aero.Coefs.s' : rocket_dict['fins_s'],
            'Rocket.Aero.Coefs.Xt' : rocket_dict['fins_Xt'],
            'Rocket.Aero.Coefs.Cr' : rocket_dict['fins_Cr'],
            'Rocket.Aero.Coefs.Ct' : rocket_dict['fins_Ct'],
            'Rocket.Aero.Coefs.tf' : rocket_dict['fins_thickness'],
            'Rocket.Aero.Coefs.delta' : rocket_dict['delta'],

            'Wind.wind_on' : rocket_dict['wind_on'],
            # 'Wind.wind_average_speed' : rocket_dict['wind_average_speed'],

            'Para.l0' : rocket_dict['parachute_l0'],
            'Para.m1' : rocket_dict['parachute_weight'] + rocket_dict['ejected_nose_mass'],
            'Para.m2' : rocket_dict['rocket_mass'] - rocket_dict['ejected_nose_mass'],
            'Para.DynPar.S_ref' : rocket_dict['parachute_sref'],
            'Para.DynPar.Cd' : rocket_dict['parachute_Cd'],

            'Traj.parachute_deploy_method' : 0 if rocket_dict['parachute_deploy_method'] == 'velocity' else 1,
            'Traj.parachute_deploy_timer' : rocket_dict['parachute_deploy_timer'],
            }

In [6]:
def run_analysis(syst, draws=10, linear=True):
    syst.drivers.clear()  # Remove all drivers on the System

    runonce = syst.add_driver(RunOnce("runonce"))
    driver = syst.add_driver(RungeKutta(order=4, dt=dt))
    driver.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'Rocket.Kin.v', 'initrot', 'Rocket.Thrust.inclinaison']), period=0.1)
    driver.add_child(NonLinearSolver('solver', factor=1.0))
    driver.time_interval = (0, 70)

    # Define a simulation scenario
    driver.set_scenario(
        init = init,
        stop = 'Para.DynPar.r2[2] < -1'
    )

    syst.run_drivers()

    return driver.recorder.export_data()

    print("MONTECARLO")
    # Montecarlo
    syst.drivers.clear()
    montecarlo = syst.add_driver(MonteCarlo('mc'))
    montecarlo.add_recorder(DataFrameRecorder(includes=['Rocket.Kin.ar', 'Traj.r', 'initrot', 'Traj.v.val', 'Rocket.Thrust.inclinaison']))
    montecarlo.add_child(driver)
    montecarlo.draws = draws
    # montecarlo.linear = linear

    # parameters for uncertainties in the data
    pitch_attr = syst.inwards.get_details('pitch_init')
    yaw_attr = syst.inwards.get_details('yaw_init')
    bang_attr = syst.Rocket.Thrust.inwards.get_details('inclinaison')
    
    # Set the distribution around the current value
    pitch_attr.distribution = Normal(best=.02, worst=-.02)
    yaw_attr.distribution = Normal(best=.02, worst=-.02)
    bang_attr.distribution = Normal(best=.01, worst=-0.01)

    montecarlo.add_random_variable('pitch_init')
    montecarlo.add_random_variable('yaw_init')
    montecarlo.add_random_variable('Rocket.Thrust.inclinaison')

    # Computation
    syst.run_drivers()

    return montecarlo.recorder.export_data()

In [7]:

results = run_analysis(earth, draws=2, linear=False)

___PARACHUTE DEPLOYMENT___


Rocket X Coordinate at Deployment:  7.48193325851587 m
Rocket Y Coordinate at Deployment:  -1.3309783342342258e-16 m
Rocket Z Coordinate at Deployment:  142.91718290553962 m


___PARACHUTE DEPLOYMENT___


Parachute fully deployed !


In [8]:
results

,Section,Status,Error code,Reference,Rocket.Kin.ar,Rocket.Kin.v,Rocket.Thrust.inclinaison,Traj.r,initrot,time
0,,,0,t=0.0,"[0.0, -1.4707963267948965, 0.0]","[0.0, 0.0, 0.0]",0.0,"[0.0, 0.0, 0.34229485]","[0.0, -1.4707963267948965, 0.0]",0.000000
1,,,0,t=0.1,"[2.1001236471184798e-20, -1.4708307250573402, ...","[9.046849540322537, -3.488856609605185e-20, -0...",0.0,"[0.07943339119140401, -1.8677422604241682e-21,...","[0.0, -1.4707963267948965, 0.0]",0.100000
2,,,0,t=0.2,"[2.295371353739012e-19, -1.471170750818207, -2...","[18.16656095484718, -2.599463172090392e-19, -0...",0.0,"[0.25478155841163963, -2.03286594436171e-20, 2...","[0.0, -1.4707963267948965, 0.0]",0.200000
3,,,0,t=0.3,"[1.0078723094714093e-18, -1.4724159136646713, ...","[26.835627348558624, -8.533069472083209e-19, -...",0.0,"[0.5242121733785214, -8.792170444280844e-20, 5...","[0.0, -1.4707963267948965, 0.0]",0.300000
4,,,0,t=0.4,"[2.9966866411496974e-18, -1.4754394140566243, ...","[35.037275086166126, -1.9518267686925606e-18, ...",0.0,"[0.8758090272944228, -2.5201756477835407e-19, ...","[0.0, -1.4707963267948965, 0.0]",0.400000
...,...,...,...,...,...,...,...,...,...,...
216,,,0,t=17.77955028288,"[-118.46844700939036, -4.80079746995903, -412....","[-5.377000421097496, -2.655767702572736e-15, -...",0.0,"[43.90088679142912, 7.829760911552642, 173.678...","[0.0, -1.4707963267948965, 0.0]",17.779550
217,,,0,Traj.VelocityPara,"[-118.46844700939036, -4.80079746995903, -412....","[-5.377000421097496, -2.655767702572736e-15, -...",0.0,"[43.90088679142912, 7.829760911552642, 173.678...","[0.0, -1.4707963267948965, 0.0]",17.779550
218,,,0,t=17.8,"[-119.05142626345207, -4.099163455160114, -412...","[-5.429240525056687, -2.6595199192308313e-15, ...",0.0,"[44.70446555526106, 6.385053978813607, 174.861...","[0.0, -1.4707963267948965, 0.0]",17.800000
219,,,0,t=17.810425036615,"[-120.60580142097866, -3.962644567997055, -412...","[-5.455871944616995, -2.66143275723324e-15, -9...",0.0,"[44.25794606022743, 5.7042570332625635, 174.88...","[0.0, -1.4707963267948965, 0.0]",17.810425


In [8]:
traj = np.asarray(results['Traj.r'].tolist())
traj

array([[ 0.00000000e+00,  0.00000000e+00,  3.42294850e-01],
       [ 2.95284376e-02, -1.81036345e-22,  6.12083033e-01],
       [ 7.94333912e-02, -1.86774226e-21,  1.06075598e+00],
       ...,
       [ 2.51358727e+01, -1.49334686e+01,  1.70176760e+02],
       [ 2.43256985e+01, -1.45745489e+01,  1.71060667e+02],
       [ 2.43256985e+01, -1.45745489e+01,  1.71060667e+02]])

In [7]:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

traj = np.asarray(results['Traj.r'].tolist())

# Create the figure object
fig = make_subplots(rows=1, cols=1)
fig.layout.title = "Probability"
fig.layout.yaxis.title = 'Y Position'

fig.add_trace(
    go.Scatter(
        x=traj[:,0],
        y=traj[:,1],
        mode = 'markers'
    ),
    row = 1,
    col = 1,
)
fig.get_subplot(1, 1).xaxis.title = "X Position"

fig.show()